Using py 3.11.0

In [6]:
import pandas as pd

In [7]:
#preparing dataset like for sarimax
df = pd.read_csv("..\\data\\processed_balance_sheet.csv")
df = df[df.columns[1:]]
df['years'] = pd.to_datetime(df['years'],infer_datetime_format=True)
df = df.set_index(['years'])
df = df['Total Current Liabilities']
df

years
2022-03-01    6813.40
2021-03-01    5080.79
2020-03-01    6547.80
2019-03-01    8132.30
2018-03-01    2868.05
2017-03-01    5464.95
2016-03-01    6488.49
2015-03-01    4346.06
2014-03-01    1869.11
2013-03-01    2977.21
2012-03-01    2789.05
2011-03-01    2285.10
2010-03-01    2772.58
2009-03-01    1447.37
2008-03-01    1157.82
2007-03-01    1053.17
2006-03-01     780.28
2005-03-01     672.83
2004-03-01     538.67
Name: Total Current Liabilities, dtype: float64

In [8]:
from numpy import asarray
from pandas import DataFrame
from pandas import concat
from xgboost import XGBRegressor
 
# transform a time series dataset into a supervised learning dataset
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    #n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols = list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
    # put it all together
    agg = concat(cols, axis=1)
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg.values
 
# load the dataset
series = df
values = series.values
# transform the time series data into supervised learning
train = series_to_supervised(values, n_in=18)
# split into input and output columns
trainX, trainy = train[:, :-1], train[:, -1]
# fit model
model = XGBRegressor(objective='reg:squarederror', n_estimators=1000)
model.fit(trainX, trainy)
# construct an input for a new preduction
row = values[-18:].flatten()
# make a one-step prediction
yhat = model.predict(asarray([row]))
print('Input: %s, Predicted: %.3f' % (row, yhat[0]))

Input: [5080.79 6547.8  8132.3  2868.05 5464.95 6488.49 4346.06 1869.11 2977.21
 2789.05 2285.1  2772.58 1447.37 1157.82 1053.17  780.28  672.83  538.67], Predicted: 538.670


In [9]:
"""series.plot(figsize=(10, 5), title='Total Current Liabilities')"""

"series.plot(figsize=(10, 5), title='Total Current Liabilities')"

In [10]:
#from pandas import DataFrame
from xgboost import XGBRegressor
import matplotlib.pyplot as plt

# Add lag features
lags = 10
for i in range(1, lags+1):
    df[f"Lag{i}"] = df["Total Current Liabilities"].shift(i)

# Split into train/test sets
train_size = int(len(df) * 0.8)
train, test = df.iloc[:train_size], df.iloc[train_size:]

# Define the XGBoost model
model = XGBRegressor(n_estimators=100, max_depth=5)

# Train the model on the training set
X_train, y_train = train.iloc[:, 1:], train.iloc[:, 0]
model.fit(X_train, y_train)

# Predict the next 10 years of total current liabilities
X_test = test.iloc[-lags:, 1:]
preds = []
for i in range(10):
    pred = model.predict(X_test.iloc[-lags:, :])
    preds.append(pred[-1])
    X_test = X_test.shift(-1)
    X_test.iloc[-1, -lags:] = pred

# Combine the original data with the predicted values
index = pd.date_range(start=df.index[-1], periods=11, freq="AS")
pred_df = pd.DataFrame(data=preds, index=index[1:], columns=["Total Current Liabilities"])
df_extended = pd.concat([df, pred_df])

# Print the predicted values
print(df_extended.tail(11)["Total Current Liabilities"])

# Plot the graph
plt.plot(df["Total Current Liabilities"], color="blue")
plt.plot(df_extended["Total Current Liabilities"], color="red")
plt.title("Total Current Liabilities")
plt.xlabel("Year")
plt.ylabel("Amount (in millions)")
plt.show()


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "pandas\_libs\tslibs\parsing.pyx", line 440, in pandas._libs.tslibs.parsing.parse_datetime_string_with_reso
  File "pandas\_libs\tslibs\parsing.pyx", line 649, in pandas._libs.tslibs.parsing.dateutil_parse
ValueError: Unknown datetime string format, unable to parse: Total Current Liabilities

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Python311\Lib\site-packages\pandas\core\indexes\datetimes.py", line 704, in get_loc
    parsed, reso = self._parse_with_reso(key)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python311\Lib\site-packages\pandas\core\indexes\datetimelike.py", line 230, in _parse_with_reso
    parsed, reso_str = parsing.parse_time_string(label, freq)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "pandas\_libs\tslibs\parsing.pyx", line 367, in pandas._libs.tslibs.parsing.parse_time_string
  File "pandas\_libs\tslibs\parsing.pyx",